In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv('C:\\Users\\DVK\\Desktop\\Amazon_decade_Sales_project\\data\\amazon_india_2015.csv')
df.head()

,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating
0,TXN_2015_00000001,2015-01-25,CUST_2015_00003884,PROD_000021,Samsung Galaxy S6 16GB Black,Electronics,Smartphones,Samsung,123614.29,27.91,...,True,Republic Day Sale,5.0,Delivered,1,2015,1,0.19,True,4.7
1,TXN_2015_00000002,2015-01-05,CUST_2015_00011709,PROD_000055,OnePlus OnePlus 2 16GB White,Electronics,Smartphones,OnePlus,54731.86,0.00,...,False,NaN,4.5,Delivered,1,2015,1,0.20,True,4.1
2,TXN_2015_00000003,2015-01-24,CUST_2015_00004782,PROD_000039,Samsung Galaxy Note 5 64GB Black,Electronics,Smartphones,Samsung,97644.25,46.93,...,True,Republic Day Sale,NaN,Delivered,1,2015,1,0.17,True,3.3
3,TXN_2015_00000004,2015-01-28,CUST_2015_00008105,PROD_000085,Motorola Moto G (3rd Gen) 16GB Black,Electronics,Smartphones,Motorola,"21,947.26",0.00,...,False,NaN,3.0,Delivered,1,2015,1,0.22,True,3.5
4,TXN_2015_00000005,2015-01-31,CUST_2015_00002955,PROD_000055,OnePlus OnePlus 2 16GB White,Electronics,Smartphones,OnePlus,54731.86,0.00,...,FALSE,NaN,4.0,Delivered,1,2015,1,0.20,True,4.1


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33165 entries, 0 to 33164
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          33165 non-null  object 
 1   order_date              33165 non-null  object 
 2   customer_id             33165 non-null  object 
 3   product_id              33165 non-null  object 
 4   product_name            33165 non-null  object 
 5   category                33165 non-null  object 
 6   subcategory             33165 non-null  object 
 7   brand                   33165 non-null  object 
 8   original_price_inr      33165 non-null  object 
 9   discount_percent        33165 non-null  float64
 10  discounted_price_inr    33165 non-null  float64
 11  quantity                33165 non-null  int64  
 12  subtotal_inr            33165 non-null  float64
 13  delivery_charges        30511 non-null  float64
 14  final_amount_inr        33165 non-null

In [5]:
df.shape


(33165, 34)

In [6]:
df['order_date'] = pd.to_datetime(
    df['order_date'], 
    errors='coerce',     # turn bad ones into NaT
    dayfirst=True,       # needed for DD-MM-YYYY and DD/MM/YYYY
    format="mixed"       # <-- NEW in Pandas 2.0, handles mixed styles
)


In [7]:
df['original_price_inr'] = (
    df['original_price_inr']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.replace('Rs', '', regex=False)
    .str.replace('-', '', regex=False)
)

df['original_price_inr'] = pd.to_numeric(
    df['original_price_inr'], errors='coerce'
)


In [8]:
df['delivery_days'].value_counts()

delivery_days
4           9892
3           9762
5           6360
6           4820
7           1667
-1           208
Same Day     134
1-2 days     128
0             70
Express       62
15            62
Name: count, dtype: int64

In [9]:
import numpy as np

df['delivery_days'] = df['delivery_days'].replace({
    'Same Day': 0,
    'Express': 1,
    '1-2 days': 2
    
})


In [10]:
df['delivery_days'] = pd.to_numeric(df['delivery_days'], errors='coerce')


In [11]:
df.loc[df['delivery_days'] <= 0, 'delivery_days'] = np.nan


In [12]:
df['delivery_days'].fillna(df['delivery_days'].median(), inplace=True)


In [13]:
df['delivery_days'].value_counts()


delivery_days
4.0     10304
3.0      9762
5.0      6360
6.0      4820
7.0      1667
2.0       128
1.0        62
15.0       62
Name: count, dtype: int64

In [14]:
df["delivery_charges"].value_counts()

delivery_charges
0.0    30511
Name: count, dtype: int64

In [15]:
df["customer_age_group"].unique()

array(['46-55', '26-35', '36-45', '18-25', nan, '55+'], dtype=object)

In [16]:
df['festival_name'].unique()

array(['Republic Day Sale', nan, 'Valentine Sale', 'Holi Festival',
       'Summer Sale', 'Back to School', 'Prime Day',
       'Amazon Great Indian Festival', 'Diwali Sale'], dtype=object)

In [17]:
df['festival_name'].fillna('No Festival', inplace=True)

In [18]:
df['customer_rating'].unique()

array(['5.0', '4.5', nan, '3.0', '4.0', '3.5', '4/5', '5.0 stars',
       '5.0/5.0', '4.5 stars', '3.5/5.0', '3/5', '4', '4.0 stars',
       '4.5/5.0', '5', '5/5', '4.0/5.0', '3.5 stars', '3.0 stars', '3',
       '3.0/5.0'], dtype=object)

In [19]:
df['customer_rating_clean'] = df['customer_rating']


In [20]:
import numpy as np

def parse_rating(value):
    if pd.isna(value):
        return np.nan
    
    value = str(value).lower().strip()
    
    # Handle fraction like 3.5/5 or 4/5
    if '/' in value:
        try:
            num, denom = value.split('/')
            return float(num) / float(denom) * 5
        except:
            return np.nan
    
    # Handle stars & plain numbers
    try:
        return float(value.replace('stars', '').strip())
    except:
        return np.nan


In [21]:
df['customer_rating_clean'] = df['customer_rating_clean'].apply(parse_rating)


In [22]:
df.loc[
    (df['customer_rating_clean'] < 1) | 
    (df['customer_rating_clean'] > 5),
    'customer_rating_clean'
] = np.nan


In [23]:
df['customer_rating_clean'].unique()

array([5. , 4.5, nan, 3. , 4. , 3.5])

In [24]:
df['delivery_charges'] = df['delivery_charges'].fillna(0)


In [25]:
df['customer_age_group'] = df['customer_age_group'].fillna('Unknown')


In [26]:
mean_rating = df['customer_rating_clean'].mean()
df['customer_rating_clean'] = df['customer_rating_clean'].fillna(mean_rating)


In [27]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33165 entries, 0 to 33164
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   transaction_id          33165 non-null  object        
 1   order_date              33165 non-null  datetime64[ns]
 2   customer_id             33165 non-null  object        
 3   product_id              33165 non-null  object        
 4   product_name            33165 non-null  object        
 5   category                33165 non-null  object        
 6   subcategory             33165 non-null  object        
 7   brand                   33165 non-null  object        
 8   original_price_inr      33165 non-null  float64       
 9   discount_percent        33165 non-null  float64       
 10  discounted_price_inr    33165 non-null  float64       
 11  quantity                33165 non-null  int64         
 12  subtotal_inr            33165 non-null  float6

In [28]:
df.isnull().sum()

transaction_id               0
order_date                   0
customer_id                  0
product_id                   0
product_name                 0
category                     0
subcategory                  0
brand                        0
original_price_inr           0
discount_percent             0
discounted_price_inr         0
quantity                     0
subtotal_inr                 0
delivery_charges             0
final_amount_inr             0
customer_city                0
customer_state               0
customer_tier                0
customer_spending_tier       0
customer_age_group           0
payment_method               0
delivery_days                0
delivery_type                0
is_prime_member              0
is_festival_sale             0
festival_name                0
customer_rating           9969
return_status                0
order_month                  0
order_year                   0
order_quarter                0
product_weight_kg            0
is_prime

In [29]:
df.shape

(33165, 35)

In [30]:
df.shape


(33165, 35)

In [31]:
df.isna().sum()


transaction_id               0
order_date                   0
customer_id                  0
product_id                   0
product_name                 0
category                     0
subcategory                  0
brand                        0
original_price_inr           0
discount_percent             0
discounted_price_inr         0
quantity                     0
subtotal_inr                 0
delivery_charges             0
final_amount_inr             0
customer_city                0
customer_state               0
customer_tier                0
customer_spending_tier       0
customer_age_group           0
payment_method               0
delivery_days                0
delivery_type                0
is_prime_member              0
is_festival_sale             0
festival_name                0
customer_rating           9969
return_status                0
order_month                  0
order_year                   0
order_quarter                0
product_weight_kg            0
is_prime

In [32]:
df.duplicated().sum()


np.int64(0)

In [33]:
df['order_date'].min(), df['order_date'].max()


(Timestamp('2015-01-01 00:00:00'), Timestamp('2015-12-31 00:00:00'))

In [34]:
df[df['order_date'].isna()]


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating,customer_rating_clean


In [35]:
df.describe()


,order_date,original_price_inr,discount_percent,discounted_price_inr,quantity,subtotal_inr,delivery_charges,final_amount_inr,delivery_days,order_month,order_year,order_quarter,product_weight_kg,product_rating,customer_rating_clean
count,33165,3.316500e+04,33165.000000,33165.000000,33165.000000,33165.000000,33165.0,33165.000000,33165.000000,33165.000000,33165.0,33165.000000,33165.000000,33165.000000,33165.000000
mean,2015-07-17 06:27:46.648574976,7.981622e+04,17.759225,51839.024371,1.246917,64591.089351,0.0,64591.089351,4.346118,7.035399,2015.0,2.681562,1.087242,3.933858,4.321564
min,2015-01-01 00:00:00,2.374840e+03,0.000000,752.200000,1.000000,752.200000,0.0,752.200000,1.000000,1.000000,2015.0,1.000000,0.030000,3.000000,3.000000
25%,2015-04-17 00:00:00,2.509387e+04,0.000000,20937.460000,1.000000,23175.030000,0.0,23175.030000,3.000000,4.000000,2015.0,2.000000,0.170000,3.600000,4.000000
50%,2015-07-24 00:00:00,4.645894e+04,11.210000,39490.870000,1.000000,45182.310000,0.0,45182.310000,4.000000,7.000000,2015.0,3.000000,0.210000,3.900000,4.321564
75%,2015-10-21 00:00:00,8.887268e+04,28.790000,73967.020000,1.000000,86591.570000,0.0,86591.570000,5.000000,10.000000,2015.0,4.000000,0.330000,4.300000,4.500000
max,2015-12-31 00:00:00,2.117212e+07,69.990000,211721.160000,3.000000,635163.480000,0.0,635163.480000,15.000000,12.000000,2015.0,4.000000,33.520000,4.800000,5.000000
std,NaN,3.823161e+05,20.690008,39768.552457,0.533607,60999.725588,0.0,60999.725588,1.287844,3.495066,0.0,1.135442,4.412278,0.462786,0.480203


In [36]:
df = df[df['quantity'] > 0]


In [38]:
df['customer_city'] = df['customer_city'].str.lower().str.strip()


In [39]:
df['customer_city'].value_counts()


customer_city
mumbai           5340
delhi            4732
bangalore        3765
chennai          3323
kolkata          2499
hyderabad        1752
pune             1444
ahmedabad        1104
surat             909
jaipur            828
nagpur            819
kanpur            751
lucknow           718
indore            662
coimbatore        547
kochi             491
chandigarh        448
bhubaneswar       418
visakhapatnam     404
patna             379
vadodara          361
ludhiana          349
saharanpur        160
meerut            132
allahabad         128
moradabad         108
gorakhpur          92
varanasi           86
aligarh            85
bareilly           61
new delhi          34
chenai             30
bengaluru          28
banglore           28
bengalore          26
mumba              26
delhi ncr          26
bombay             25
calcutta           25
madras             22
Name: count, dtype: int64

In [40]:
(df['discounted_price_inr'] <= df['original_price_inr']).all()


np.False_

In [42]:
invalid_prices = df[df['discounted_price_inr'] > df['original_price_inr']]
invalid_prices


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating,customer_rating_clean
76,TXN_2015_00000077,2015-01-13,CUST_2015_00005251,PROD_000104,Motorola Moto E (2nd Gen) 32GB White,Electronics,Smartphones,Motorola,24762.0,0.0,...,No Festival,3.5,Delivered,1,2015,1,0.23,1,3.8,3.500000
287,TXN_2015_00000288,2015-01-29,CUST_2015_00003918,PROD_000092,Motorola Moto G (3rd Gen) 16GB Gold,Electronics,Smartphones,Motorola,17869.0,0.0,...,No Festival,3.0,Delivered,1,2015,1,0.24,True,4.0,3.000000
394,TXN_2015_00000395,2015-01-31,CUST_2015_00004912,PROD_001943,Garmin Sports Watch Premium,Electronics,Smart Watch,Garmin,45182.0,0.0,...,No Festival,NaN,Returned,1,2015,1,0.07,True,4.2,4.321564
517,TXN_2015_00000518,2015-01-08,CUST_2015_00009268,PROD_000092,Motorola Moto G (3rd Gen) 16GB Gold,Electronics,Smartphones,Motorola,17869.0,0.0,...,No Festival,4.0,Delivered,1,2015,1,0.24,True,4.0,4.000000
535,TXN_2015_00000536,2015-01-09,CUST_2015_00000635,PROD_001692,Samsung Galaxy Tab 8GB RAM Silver,Electronics,Tablets,Samsung,77250.0,0.0,...,No Festival,5.0,Delivered,1,2015,1,0.43,False,4.8,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32174,TXN_2015_00032175,2015-12-18,CUST_2015_00010295,PROD_000021,Samsung Galaxy S6 16GB Black,Electronics,Smartphones,Samsung,123614.0,0.0,...,No Festival,4.5,Delivered,12,2015,4,0.19,True,4.7,4.500000
32248,TXN_2015_00032249,2015-12-14,CUST_2015_00001528,PROD_001983,Samsung OLED TV,Electronics,TV & Entertainment,Samsung,38884.0,0.0,...,No Festival,NaN,Delivered,12,2015,4,25.87,False,3.9,4.321564
32263,TXN_2015_00032264,2015-12-21,CUST_2015_00005054,PROD_001983,Samsung OLED TV,Electronics,TV & Entertainment,Samsung,38884.0,0.0,...,No Festival,NaN,Delivered,12,2015,4,25.87,False,3.9,4.321564
32331,TXN_2015_00032332,2015-12-12,CUST_2015_00004112,PROD_000078,Xiaomi Redmi Note 4G 16GB Black,Electronics,Smartphones,Xiaomi,13344.0,0.0,...,No Festival,4.0,Delivered,12,2015,4,0.24,True,4.3,4.000000


In [43]:
invalid_prices.shape[0]


202

In [44]:
total_rows = df.shape[0]
invalid_count = invalid_prices.shape[0]

(invalid_count / total_rows) * 100


0.60907583295643

In [45]:
invalid_prices[['original_price_inr',
                'discounted_price_inr',
                'discount_percent']].describe()


,original_price_inr,discounted_price_inr,discount_percent
count,202.000000,202.000000,202.0
mean,63204.153465,63204.396832,0.0
std,49728.165754,49728.162810,0.0
min,3795.000000,3795.010000,0.0
25%,23391.000000,23391.490000,0.0
50%,44830.000000,44830.400000,0.0
75%,101397.750000,101397.917500,0.0
max,211721.000000,211721.160000,0.0


In [46]:
invalid_prices['discount_percent'].value_counts().head(10)


discount_percent
0.0    202
Name: count, dtype: int64

In [47]:
df['discounted_price_inr'] = (
    df['original_price_inr'] * (1 - df['discount_percent'] / 100)
).round(2)


In [48]:
(df['discounted_price_inr'] <= df['original_price_inr']).all()


np.True_

In [49]:
checks = {
    "no_missing_values": df.isna().sum().sum() == 0,
    "no_duplicates": df.duplicated().sum() == 0,
    "valid_prices": (df['discounted_price_inr'] <= df['original_price_inr']).all(),
    "positive_quantity": (df['quantity'] > 0).all(),
    "valid_ratings": df['product_rating'].between(1, 5).all(),
    "non_negative_delivery": (df['delivery_days'] >= 0).all()
}

checks


{'no_missing_values': np.False_,
 'no_duplicates': np.True_,
 'valid_prices': np.True_,
 'positive_quantity': np.True_,
 'valid_ratings': np.True_,
 'non_negative_delivery': np.True_}

In [50]:
df.isna().sum().sort_values(ascending=False)


customer_rating           9969
order_date                   0
transaction_id               0
customer_id                  0
product_id                   0
category                     0
product_name                 0
brand                        0
original_price_inr           0
discount_percent             0
subcategory                  0
quantity                     0
subtotal_inr                 0
delivery_charges             0
final_amount_inr             0
customer_city                0
customer_state               0
customer_tier                0
discounted_price_inr         0
customer_spending_tier       0
customer_age_group           0
delivery_days                0
payment_method               0
delivery_type                0
is_prime_member              0
is_festival_sale             0
festival_name                0
return_status                0
order_month                  0
order_year                   0
order_quarter                0
product_weight_kg            0
is_prime

In [51]:
df['delivery_days'] = df['delivery_days'].fillna(df['delivery_days'].median())


In [52]:
df['customer_age_group'] = df['customer_age_group'].fillna('Unknown')


In [53]:
df = df.dropna(subset=['order_date'])


In [54]:
df.isna().sum()


transaction_id               0
order_date                   0
customer_id                  0
product_id                   0
product_name                 0
category                     0
subcategory                  0
brand                        0
original_price_inr           0
discount_percent             0
discounted_price_inr         0
quantity                     0
subtotal_inr                 0
delivery_charges             0
final_amount_inr             0
customer_city                0
customer_state               0
customer_tier                0
customer_spending_tier       0
customer_age_group           0
payment_method               0
delivery_days                0
delivery_type                0
is_prime_member              0
is_festival_sale             0
festival_name                0
customer_rating           9969
return_status                0
order_month                  0
order_year                   0
order_quarter                0
product_weight_kg            0
is_prime

In [55]:
checks = {
    "no_missing_values": df.isna().sum().sum() == 0,
    "no_duplicates": df.duplicated().sum() == 0,
    "valid_prices": (df['discounted_price_inr'] <= df['original_price_inr']).all(),
    "positive_quantity": (df['quantity'] > 0).all(),
    "valid_ratings": df['product_rating'].between(1, 5).all(),
    "non_negative_delivery": (df['delivery_days'] >= 0).all()
}

checks


{'no_missing_values': np.False_,
 'no_duplicates': np.True_,
 'valid_prices': np.True_,
 'positive_quantity': np.True_,
 'valid_ratings': np.True_,
 'non_negative_delivery': np.True_}

In [58]:
df['is_rating_missing'] = df['customer_rating'].isna().astype(int)


In [59]:
df.isna().sum().sum() == 0


np.True_

In [60]:
checks = {
    "no_missing_values": df.isna().sum().sum() == 0,
    "no_duplicates": df.duplicated().sum() == 0,
    "valid_prices": (df['discounted_price_inr'] <= df['original_price_inr']).all(),
    "positive_quantity": (df['quantity'] > 0).all(),
    "valid_ratings": df['product_rating'].between(1, 5).all(),
    "non_negative_delivery": (df['delivery_days'] >= 0).all()
}

checks


{'no_missing_values': np.True_,
 'no_duplicates': np.True_,
 'valid_prices': np.True_,
 'positive_quantity': np.True_,
 'valid_ratings': np.True_,
 'non_negative_delivery': np.True_}

In [63]:
df.shape

(33165, 36)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33165 entries, 0 to 33164
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   transaction_id          33165 non-null  object        
 1   order_date              33165 non-null  datetime64[ns]
 2   customer_id             33165 non-null  object        
 3   product_id              33165 non-null  object        
 4   product_name            33165 non-null  object        
 5   category                33165 non-null  object        
 6   subcategory             33165 non-null  object        
 7   brand                   33165 non-null  object        
 8   original_price_inr      33165 non-null  float64       
 9   discount_percent        33165 non-null  float64       
 10  discounted_price_inr    33165 non-null  float64       
 11  quantity                33165 non-null  int64         
 12  subtotal_inr            33165 non-null  float6

In [194]:
df["product_rating"].sample(5)

8944     4.3
8800     3.8
21059    4.4
27518    4.0
29767    3.7
Name: product_rating, dtype: float64

In [64]:
df = df.drop(columns=['customer_rating'])


In [195]:
df.to_csv("amazon_2015_cleaned.csv", index=False)


In [196]:
df.isnull().sum()

transaction_id            0
order_date                0
customer_id               0
product_id                0
product_name              0
category                  0
subcategory               0
brand                     0
original_price_inr        0
discount_percent          0
discounted_price_inr      0
quantity                  0
subtotal_inr              0
delivery_charges          0
final_amount_inr          0
customer_city             0
customer_state            0
customer_tier             0
customer_spending_tier    0
customer_age_group        0
payment_method            0
delivery_days             0
delivery_type             0
is_prime_member           0
is_festival_sale          0
festival_name             0
return_status             0
order_month               0
order_year                0
order_quarter             0
product_weight_kg         0
is_prime_eligible         0
product_rating            0
customer_rating_clean     0
is_rating_missing         0
dtype: int64